# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 1 : Propiedades en Venta</center>
### <center> Grupo 22 </center>
### <center> Integrantes: </center>
##### <center> Federico Martin Forte,  Francisco Sobral,  Ian Klaus von der Heyde,  Juan Pablo Aschieri,  Joaquin Rivero  </center>






## Carga de Datos y librerías

In [54]:
import pandas as pd
import numpy as np

import sklearn as sk

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('always')

In [55]:
dataset=pd.read_csv('./Datasets/properati_argentina_2021.csv')

## Análisis Exploratorio y Preprocesamiento de Datos

Exploración Inicial :

In [56]:
dataset.head()

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
0,XZq2dFGxBiboHY3Bo/tqlw==,2021-09-09,2022-01-27,2021-09-09,-38.957632,-68.105443,Neuquén,NaN,NaN,NaN,NaN,Venta,Casa,6.0,NaN,NaN,206.0,NaN,NaN,Casa - Capital
1,8afoK+yMH/rnnlYiktwgNA==,2021-09-09,2021-10-23,2021-09-09,-34.888878,-56.092368,Montevideo,NaN,NaN,NaN,NaN,Venta,Casa,4.0,3.0,193.0,193.0,390000.0,USD,Casa - Malvín
2,iVaG5rBB47dd1Ya4BMXREA==,2021-09-09,2022-03-09,2021-09-09,-32.488717,-58.257499,Entre Ríos,NaN,NaN,NaN,NaN,Venta,Casa,NaN,NaN,110.0,45.0,47000.0,USD,Casa - Uruguay
3,QE1QVMWKAHWkd0qT0FtPWA==,2021-09-09,2022-01-27,2021-09-09,-34.900165,-54.808453,Maldonado,NaN,NaN,NaN,NaN,Venta,Casa,NaN,3.0,NaN,180.0,400000.0,USD,Casa - El Chorro
4,tJ5lSk7IysG/71OGhacNZA==,2021-09-09,2022-01-27,2021-09-09,-38.954528,-68.041862,Neuquén,NaN,NaN,NaN,NaN,Venta,PH,3.0,2.0,NaN,98.0,102000.0,USD,PH - Villa Farrell


In [64]:
dataset.isna().sum()

id                               0
start_date                       0
end_date                         0
created_on                       0
latitud                      40414
longitud                     40414
place_l2                         0
place_l3                     22489
place_l4                    321134
place_l5                    457724
place_l6                    460154
operation                        0
property_type                    0
property_rooms               91656
property_bedrooms           116041
property_surface_total       62341
property_surface_covered     32238
property_price               18001
property_currency            18564
property_title                   0
dtype: int64

In [47]:
viviendas = dataset[dataset.property_type.isin(['Casa', 'PH', 'Departamento', 'Casa de campo'])]  #deberiamos sacar la casa de campo?
viviendas_caba= viviendas[viviendas.place_l2 == 'Capital Federal']
viviendas_en_venta_caba = viviendas_caba[viviendas_caba.operation == 'Venta']
viviendas_en_venta_caba_usd = viviendas_en_venta_caba[viviendas_en_venta_caba.property_currency == 'USD']
viviendas_en_venta_caba_usd.head()

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
25,CyZu6IaZgmQYGk8jmPajCg==,2021-09-09,2021-09-26,2021-09-09,NaN,NaN,Capital Federal,NaN,NaN,NaN,NaN,Venta,Casa,4.0,3.0,171.0,133.0,210000.0,USD,VENTA CASA LLAVE EN MANO - COSTA ESMERALDA
42,d3Z0MblSb2IMBoOBMGJEEQ==,2021-09-09,2021-09-26,2021-09-09,-34.637319,-58.361124,Capital Federal,Boca,NaN,NaN,NaN,Venta,Departamento,3.0,2.0,58.0,55.0,82900.0,USD,VENTA TRES AMBIENTES LA BOCA PISO ALTO
43,rL7qPkQnomsMGu2ViS3sBw==,2021-09-09,2021-09-28,2021-09-09,-34.628647,-58.361871,Capital Federal,Boca,NaN,NaN,NaN,Venta,Departamento,4.0,3.0,70.0,70.0,99000.0,USD,Departamento - 4 Ambientes - Catalinas Sur
49,vuX/ccLAMVml4qWRFcOJUA==,2021-09-09,2022-01-05,2021-09-09,-34.610027,-58.401934,Capital Federal,Once,NaN,NaN,NaN,Venta,Departamento,4.0,2.0,NaN,104.0,146000.0,USD,Venta departamento 4 ambientes Once
51,ZTtSCeqZ3HUgf5jjwEzRcw==,2021-09-09,2022-03-09,2021-09-09,-34.631621,-58.416825,Capital Federal,Boedo,NaN,NaN,NaN,Venta,PH,4.0,2.0,NaN,73.0,250000.0,USD,PH - Boedo


In [67]:
viviendas_en_venta_caba[viviendas_en_venta_caba.property_currency.isna()] #Los que no tienen precio o tipo de moneda deberiamos borrarlos?

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
44,IfZHM0GX583/RU9QQb8Orw==,2021-09-09,2022-03-09,2021-09-09,-34.599506,-58.404138,Capital Federal,Once,NaN,NaN,NaN,Venta,Departamento,2.0,1.0,43.0,43.0,NaN,NaN,Av Puyrredon al 924
663,M9CDjvm3nBR1ECCd+0k6jw==,2021-09-09,2021-10-01,2021-09-09,-34.552724,-58.464948,Capital Federal,Belgrano,NaN,NaN,NaN,Venta,Departamento,2.0,2.0,60.0,50.0,NaN,NaN,EXCELENTE 2 AMBIENTES REFACCIONADO FRENTE BALC...
3868,o7PdJAiux9ibzOqpXNdoCA==,2021-02-20,2021-03-18,2021-02-20,-34.626710,-58.448553,Capital Federal,Caballito,NaN,NaN,NaN,Venta,Departamento,3.0,2.0,65.0,65.0,NaN,NaN,Excelente Semipiso 3 Amb c/Baulera. Caballito
3944,wYj0T1lIRh3D4tmsM/DDHw==,2021-02-20,9999-12-31,2021-02-20,-34.597851,-58.410105,Capital Federal,Barrio Norte,NaN,NaN,NaN,Venta,Departamento,3.0,NaN,141.0,59.0,NaN,NaN,Recoleta A Estrenar Tres Ambientes con Gran Ba...
4846,tMEtNCWjEV3SorEYYm1btw==,2021-09-20,2021-10-09,2021-09-20,-34.581687,-58.422602,Capital Federal,Palermo,NaN,NaN,NaN,Venta,Departamento,2.0,1.0,61.0,56.0,NaN,NaN,Departamento en Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459740,9zdu3PDJGgtkPhGdhwkNeQ==,2021-01-05,2021-02-25,2021-01-05,-34.591558,-58.379402,Capital Federal,Recoleta,NaN,NaN,NaN,Venta,Departamento,5.0,2.0,201.0,166.0,NaN,NaN,Excelente piso alto en venta con vista al rio ...
459769,/sXuWfMBUOrKetNkXdnhvA==,2021-01-05,9999-12-31,2021-01-05,-34.591558,-58.379402,Capital Federal,Recoleta,NaN,NaN,NaN,Venta,Departamento,5.0,2.0,201.0,166.0,NaN,NaN,Excelente piso alto en venta con vista al rio ...
459775,+YhiZz8j4asUVA/yEBMXCw==,2021-01-05,2022-06-11,2021-01-05,-34.592054,-58.384274,Capital Federal,Recoleta,NaN,NaN,NaN,Venta,Departamento,10.0,5.0,508.0,492.0,NaN,NaN,Departamento en edificio de excelente categorí...
459911,ujOktAtslW9dNVwNkobXDw==,2021-01-05,2022-06-02,2021-01-05,-34.597850,-58.388226,Capital Federal,Barrio Norte,NaN,NaN,NaN,Venta,Departamento,9.0,5.0,318.0,318.0,NaN,NaN,Excelente departamento en edificio frances de ...
